In [1]:
'''
Homework 5
Mass Profile 
Farah Fauzi
Feb24
'''

'\nHomework 5\nMass Profile \nFarah Fauzi\nFeb24\n'

In [1]:
# import modules
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import matplotlib

from astropy import constants as const
from operator import add
from ReadFile import Read
from centerofmass_redo import CenterOfMass

Separation between the MW and M31 = 769.0 kpc
Relative Velocity between the MW and M31 = 118.0 km / s
Relative Position between M33 and M31 =  201.0 kpc
Relative Velocity between M33 and M31 =  199.0 km / s


In [8]:
#create class MassProfile to calculate:
#  Mass Enclosed, Mass Enclosed TOtal in an array of radii
#  Circular Velocity, total CIrcular velocity of galaxy
#  Hernquist mass, Hernquist circular velocity
class MassProfile:
    
    def __init__(self, galaxy, snap):
        # Inputs:
        #    filename = galaxy name string
        #    snap = snapshot number
        
        # Determine Filename
        # add a string of the filenumber to the value "000"
        ilbl = '000' + str(snap)
        # remove all but the last 3 digits
        ilbl = ilbl[-3:]
        # create a new filename for the galaxy
        self.filename = "%s_"%(galaxy) + ilbl + '.txt'
        
        #using the function from ReadFile to read data from file
        self.time, self.total, self.data = Read(self.filename)
        
        #store mass, positions, velocities of all particles
        self.x = self.data['x']
        self.y = self.data['y']
        self.z = self.data['z']
        self.m = self.data['m']
        #store galaxy name
        self.gname = galaxy
        
    def MassEnclosed(self, ptype, r):
        # Function that determines the MassEnclosed of particles of a given type
        # Inputs:
        #    ptype = type of the particle in galaxy (1=Halo, 2=Disk, 3=Bulge)
        #    r = array of radii
        # Returns:
        #    mass array of the enclosed radius (Msun)
        
        # Determine the COM position using Disk Particles
        # Disk Particles afford the best centroiding.
        # Create a COM object
        COMObject = CenterOfMass(self.filename,ptype)
        # Store the COM position of the galaxy
        # Set Delta = whatever you determined to be a good value in Homework 4.
        COMP = COMObject.COM_P(0.1)
        #print(COM)
        #for desired ptype
        index = np.where(self.data['type'] == ptype)
        
        mass = self.m[index]
        for i in r:
            massArr = np.zeros(len(self.x))
            for j in range(len(r)):
                if ( abs(COMP[0]-self.x[index][j]*u.kpc) < i*u.kpc and 
                    abs(COMP[1]-self.y[index][j]*u.kpc) < i*u.kpc and 
                    abs(COMP[2]-self.z[index][j]*u.kpc) < i*u.kpc):
                    massArr[i] = np.sum(mass[j])
                   
        return massArr*u.Msun
        
    def MassEnclosedTotal(self,r):
        # Inputs:
        #    r = array of radii 
        # Returns:
        #    totMass = array of total mass of components in a galaxy (Msun*1e10)
        totMassEach = []
        totMass = []
        for i in range (1,4):
            mass = self.MassEnclosed(i,r)
            totMassEach.append(mass)
        totMass = list(map(add, totMassEach[0], totMassEach[1]))
        if (self.gname != "M33"):
            totMass = list(map(add, totMass, totMassEach[2]))
        
        return totMass
        
    def HernquistMass(self,r,a,Mhalo):
        # Inputs:
        #    r = array of radii 
        #.   a = scale radius
        #.   Mhalo = array of mass of halo enclosed in radii (1e10*Msun)
        # Returns: 
        #    Mass of Dark Matter in a chosen galaxy
        Mhalo = self.MassEnclosed(1,r)
        HernMass = (Mhalo*(r**2))/((a+r)**2)
        
        return HernMass
    
    def CircularVelocity(self,ptype,r):
        # Inputs:
        #    ptype = type of component in galaxy
        #    r = array of radii (kpc)
        # Returns:
        #    using formula: v = (G*M/r)^1/2
        #    circular velocity, v (km/s)
        Grav = const.G
        G = Grav.to(u.kpc*u.km**2/u.s**2/u.Msun)
        M = self.MassEnclosed(ptype,r)
        
        CircVel = np.sqrt(G*M/(r*u.kpc))
        
        #make sure dpt array of velocity
        return CircVel
    
    def CircularVelocityTotal(self,r):
        #Inputs:
        #  r = array of radii
        #Returns:
        #  using formula: vtot = (v[1]^2+v[2]^2+v[3]^2)^1/2
        #  total circular velocity of all of the component in galaxy
        totVelEach = []
        totVel = []
        for i in range(1,4):
            v = self.CircularVelocity(i,r)
            totVelEach.append(v)
        totVel = [np.sqrt(i*i + j*j + k*k) for i,j,k in zip(totVelEach[0],totVelEach[1],totVelEach[2])]
        
        #make sure right array
        return totVel
    
    def HernquistVCirc(self,r,a,Mhalo):
        # Inputs:
        #    r = array of radii
        #.   a = scale radius
        #    Mhalo = masses enclosed in radii
        # Returns:
        #    Hernquist circular velocity using mass of halo of galaxy
        Grav = const.G
        G = Grav.to(u.kpc*u.km**2/u.s**2/u.Msun)
        M = self.HernquistMass(r, a, Mhalo)
        
        HernVCirc = np.sqrt(G*M/(r*u.kpc))
        
        return HernVCirc
        

In [ ]:
# Function for plotting mass profile of galaxy
def MPPlot(galaxy,r):
    fig = plt.figure(figsize=(7,7))
    ax = plt.subplot(111)
    ax.semilogy(r,galaxy.MassEnclosed(1,r),'b^', label='halo')
    ax.semilogy(r,galaxy.MassEnclosed(2,r),'g^', label='disk')
    if (galaxy != "M33"):
        ax.semilogy(r,galaxy.MassEnclosed(3,r),'r^', label='bulge')
    ax.semilogy(r,galaxy.MassEnclosedTotal(r),'m-', label='total')
    
    #finding the a value that correspond to the mass profile
    a = findA(galaxy,r)
    ax.semilogy(r,galaxy.HernquistMass(r,a,comp1),'y-',label='hernquist')
    
    plt.xlim([0.20,30.5])

    plt.xlabel('r', fontsize=22)
    plt.ylabel('Mass Enclosed (Msun$^1e10$)', fontsize=22)
    label_size = 22
    matplotlib.rcParams['xtick.labelsize'] = label_size 
    matplotlib.rcParams['ytick.labelsize'] = label_size
    legend = ax.legend(loc='lower right',fontsize='large')
    
    plt.show()

In [ ]:
# Function for plotting Rotation Curve of galaxy
def RCPlot(galaxy,r):
    fig = plt.figure(figsize=(7,7))
    ax = plt.subplot(111)
    ax.semilogy(r,galaxy.CircularVelocity(1,r),'b^', label='halo')
    ax.semilogy(r,galaxy.CircularVelocity(2,r),'g^', label='disk')
    if (galaxy != "M33"):
        ax.semilogy(r,galaxy.CircularVelocity(3,r),'r^', label='bulge')
    ax.semilogy(r,galaxy.CircularVelocityTotal(r),'m-', label='total')
    
    #finding the a value that correspond to the mass profile
    a = findA(galaxy,r) 
    ax.semilogy(r,galaxy.HernquistVCirc(r,a,comp1),'y-',label='hernquist')
        
    plt.xlabel('r', fontsize=22)
    plt.ylabel('Circular Velocity (km/s)', fontsize=22)
    label_size = 22
    matplotlib.rcParams['xtick.labelsize'] = label_size 
    matplotlib.rcParams['ytick.labelsize'] = label_size
    legend = ax.legend(loc='lower right',fontsize='large')
    
    plt.show()

In [ ]:
def findA(galaxy,r):
    comp1 = galaxy.MassEnclosed(1,r)
    comp2 = galaxy.MassEnclosed(2,r)
    comp3 = galaxy.MassEnclosed(3,r)
    
    newComp = ( comp1[len(comp1)-1] + comp2[len(comp2)-1] + comp3[len(comp3)-1] ) / 3
    a = np.sqrt(( (comp1[len(comp1)-1])*((r[len(r)])**2) )/newComp) - r[len(r)] 
    
    return a

In [9]:
MW = MassProfile("MW",0)
M31 = MassProfile("M31",0)
M33 = MassProfile("M33",0)

r = np.arange(0.25,30.5,1.5)
print(r)

# initialize constant to be any number
a = 1
Mhalo = 1
ptype = 1

print(MW.MassEnclosed(1,r))
#print(MW.MassEnclosedTotal(r))
#print(MW.HernquistMass(r,a,Mhalo))
#print(MW.CircularVelocity(ptype,r))
#print(MW.CircularVelocityTotal(r))
#print(MW.HernquistVCirc(r,a,Mhalo))

[ 0.25  1.75  3.25  4.75  6.25  7.75  9.25 10.75 12.25 13.75 15.25 16.75
 18.25 19.75 21.25 22.75 24.25 25.75 27.25 28.75 30.25]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
galaxies = [MW,M31,M33]

# plot for Mass Profile for MW, M31 and M33

for galaxy in galaxies:
    MPPlot(galaxy,r)

    
# plot for Rotation Curve for MW, M31 and M33
'''
for galaxy in galaxies:
    RCPlot(galaxy,r)
'''